In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("Reviews.csv")

In [4]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


## Testing the loading times


In [1]:
%%timeit
import pandas as pd
pd.read_csv("Reviews.csv")

3.76 s ± 78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
%%timeit
import csv
with open('Reviews.csv', encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        data = [row for row in reader]

5.27 s ± 72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Experiment 1: Looking for chocolate
Goal of this experiment is to get the reviewscores for all reviews containg the word Chocolate (or any other words) in a list.

#### Python standard

In [1]:
with open('Reviews.csv', encoding="utf-8") as csvfile:
        import csv
        reader = csv.reader(csvfile)
        data = [row for row in reader]

In [2]:
def python_standard_choc_1(data):
    import csv

    def parse_row_chocolate(row, word):
        if word in row[9].lower():
            return row[6]

    total = []
        
    for row in data:
        result = parse_row_chocolate(row, "chocolate")
        if result:
            total.append((result))
    return len(total)

In [3]:
%%timeit
python_standard_choc_1(data)

559 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def python_standard_choc_2(data):
    import csv

    def parse_row_chocolate(row, word):
        if word in row[9].lower():
            return row[6]

    return len([x for x in map(lambda row: parse_row_chocolate(row, "chocolate"), data) if x])

In [3]:
%%timeit
python_standard_choc_2(data)

607 ms ± 8.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def python_standard_choc_3(data):
    import csv
    return len([row[6] for row in data if "chocolate" in row[9].lower()])

In [3]:
%%timeit
python_standard_choc_3(data)

487 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Pandas

In [1]:
import pandas as pd
df_load = pd.read_csv("Reviews.csv")

In [2]:
def pandas_choc_1(df):
    return len(list(df[df.Text.str.contains("chocolate", case=False, regex=False, na=False)].Score))

In [4]:
%%timeit
pandas_choc_1(df_load)

854 ms ± 3.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def pandas_choc_2(df):
    total = []
    for index, row in df.iterrows():
        if "chocolate" in row.Text.lower():
            total.append(row["Score"])
    return len(total)

In [4]:
%%timeit
pandas_choc_2(df_load)

56.7 s ± 506 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def pandas_choc_3(df):    
    total = []
    for row in df.itertuples():
        if "chocolate" in row.Text.lower():
            total.append(row.Score)
    return len(total)

In [3]:
%%timeit
pandas_choc_3(df_load)

1.42 s ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def pandas_choc_4(df):    
    def parse_chocolate_col(col):
        if "chocolate" in col.lower():
            return True
        return False
    return len(list(df[df["Text"].apply(lambda col: parse_chocolate_col(col))]['Score']))

In [4]:
%%timeit
pandas_choc_4(df_load)

600 ms ± 8.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def pandas_choc_5(df):   
    def parse_chocolate_col(col):
        if "chocolate" in col.lower():
            return True
        return False
    return len(list(df[df["Text"].map(lambda col: parse_chocolate_col(col))]['Score']))

In [3]:
%%timeit
pandas_choc_5(df_load)

604 ms ± 3.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def pandas_choc_6(df):
    df["succes"] = df.Text.str.lower().str.contains("chocolate")
    return df.succes

In [4]:
%%timeit
pandas_choc_6(df_load)

1.16 s ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numpy

In [3]:
def numpy_choc_1(df):
    import numpy as np
    def testfunc(arr):
        if "chocolate" in arr.lower():
            return True
        return False

    tfunc = np.vectorize(testfunc)
    return len(list(df[tfunc(df.Text)].Score))

In [4]:
%%timeit
numpy_choc_1(df_load)

517 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def numpy_choc_2(df):
    import numpy as np
    def testfunc(arr):
        if "chocolate" in arr.lower():
            return True
        return False

    tfunc = np.vectorize(testfunc)
    return len(list(df[tfunc(df.Text.values)].Score))

In [3]:
%%timeit
numpy_choc_2(df_load)

516 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numba

In [119]:
from numba import jit
import numpy as np

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


In [2]:
from numba import jit
# import csv

@jit
def testfunc(arr):
        if "chocolate" in arr.lower():
            return True
        return False

In [3]:
def numba_choc_1(df=None, df_bool=False):
    import numpy as np
    
    tfunc = np.vectorize(testfunc)
    return len(list(df[tfunc(df.Text.values)].Score))


In [5]:
%%timeit
numba_choc_1(df_load)

5.19 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cython

In [3]:
%load_ext Cython

In [123]:
%%cython --annotate
cdef int a = 0
for i in range(10):
    a += i
print(a)

45


In [ ]:
%load_ext Cython

with open('Reviews.csv', encoding="utf-8") as csvfile:
        import csv
        reader = csv.reader(csvfile)
        data = [row for row in reader]

In [3]:
%%cython
cpdef str parse_row_chocolate(list row, str word):
        if word in row[9].lower():
            return row[6]

cpdef int cython_standard_choc_1(list data):        
    cdef list total = []
        
    for row in data:
        result = parse_row_chocolate(row, "chocolate")
        if result:
            total.append((result))
    return len(total)

In [4]:
%%timeit
cython_standard_choc_1(data)

464 ms ± 6.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
%%cython
cpdef int cython_standard_choc_3(list data):
    return len([row[6] for row in data if "chocolate" in row[9].lower()])

In [3]:
%%timeit
cython_standard_choc_3(data)

466 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%cython
# Bint stands for boolean int
cpdef bint testfunc(str arr):
        if "chocolate" in arr.lower():
            return True
        return False

cpdef int cython_numpy_choc_1(df):
    import numpy as np
    tfunc = np.vectorize(testfunc)
    return len(list(df[tfunc(df.Text.values)].Score))

In [5]:
%%timeit
cython_numpy_choc_1(df_load)

480 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Experiment 2: Liking what?

Goal of the experiment is to calculate the std deviation of the ratings.
- Convert string to int
- Calculate the sum

#### Regular Python

In [1]:
with open('Reviews.csv', encoding="utf-8") as csvfile:
    import csv
    reader = csv.reader(csvfile)
    next(reader)
    data = [row for row in reader]

In [3]:
def python_standard_score_1(reader):
    import math

    total = []

    for row in reader:
        total.append(int(row[6]))

    average = sum(total) / len(total)
    total2 = []
    for value in total:
        total2.append((value - average) ** 2)

    std_dev = math.sqrt((1 / (len(total2) -1) * sum(total2)))

    return std_dev

In [4]:
%%timeit
python_standard_score_1(data)

356 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def python_standard_score_2(reader):
    import math

    def calc_difference(value, average):
        return (value - average) ** 2
        
    scores = [int(row[6]) for row in reader]
    average = sum(scores) / len(scores)

    deviations = list(map(lambda score: calc_difference(score, average), scores))       
    return math.sqrt((1 / (len(deviations) - 1) * sum(deviations)))

In [4]:
%%timeit
python_standard_score_2(data)

431 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def python_standard_score_2B(reader):
    import math

    def calc_difference(value, average):
        return (value - average) ** 2
    
    scores = [int(row[6]) for row in reader]
    average = sum(scores) / len(scores)

    deviations = map(lambda score: calc_difference(score, average), scores)      
    return math.sqrt((1 / (len(scores) - 1) * sum(deviations)))

In [3]:
%%timeit
python_standard_score_2B(data)

396 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
## Interesting find
def python_standard_score_3(reader):
    import statistics
        
    scores = [int(row[6]) for row in reader]
    return statistics.stdev(scores)

In [6]:
%%timeit
python_standard_score_3(data)

1.97 s ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def python_standard_score_4(reader):
    import math
    
    scores = [int(row[6]) for row in reader]
    average = sum(scores) / len(scores)

    deviations = [(value - average) ** 2 for value in scores]      
    return math.sqrt((1 / (len(deviations) - 1) * sum(deviations)))

In [3]:
%%timeit
python_standard_score_4(data)

313 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [214]:
data[0]

['1',
 'B001E4KFG0',
 'A3SGXH7AUHU8GW',
 'delmartian',
 '1',
 '1',
 '5',
 '1303862400',
 'Good Quality Dog Food',
 'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.']

#### Pandas

In [1]:
import pandas as pd
df = pd.read_csv("Reviews.csv")

In [2]:
def pandas_score_1(df): 
    import math
    
    deviations = (df.Score - df.Score.mean()) ** 2     
    return math.sqrt((1 / (len(deviations) - 1) * sum(deviations)))

In [3]:
%%timeit
pandas_score_1(df)

54.5 ms ± 408 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
def pandas_score_2(df): 
    import math
    
    deviations = (df.Score - df.Score.mean()) ** 2     
    return math.sqrt((1 / (len(deviations) - 1) * deviations.sum()))

In [3]:
%%timeit
pandas_score_2(df)

9.49 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
def pandas_score_3(df):
    return df.Score.std()

In [3]:
%%timeit
pandas_score_3(df)

7.33 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Numpy

In [2]:
def numpy_score_1(df):
    return df.Score.values.std()

In [3]:
%%timeit
numpy_score_1(df)

3.4 ms ± 49.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
def numpy_score_2(df): 
    import math
    
    deviations = (df.Score.values - df.Score.values.mean()) ** 2     
    return math.sqrt((1 / (len(deviations) - 1) * deviations.sum()))

In [5]:
%%timeit
numpy_score_2(df)

5.36 ms ± 35.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Numba

In [1]:
with open('Reviews.csv', encoding="utf-8") as csvfile:
    import csv
    reader = csv.reader(csvfile)
    next(reader)
    data = [row for row in reader]

In [3]:
from numba import jit

In [4]:
import math

@jit
def numba_score_1(reader):
    
    scores = [int(row[6]) for row in reader]
    average = sum(scores) / len(scores)

    deviations = [(value - average) ** 2 for value in scores]      
    return math.sqrt((1 / (len(deviations) - 1) * sum(deviations)))

In [5]:
%%timeit
numba_score_1(data)

<ipython-input-4-f96871e94303>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "numba_score_1" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-4-f96871e94303>", line 7:
def numba_score_1(reader):
    <source elided>
    scores = [int(row[6]) for row in reader]
    average = sum(scores) / len(scores)
    ^

  @jit
c:\users\svw-b\envs\big_data\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "numba_score_1" was compiled in object mode without forceobj=True.

File "<ipython-input-4-f96871e94303>", line 4:
@jit
def numba_score_1(reader):
^

  state.func_ir.loc))
c:\users\svw-b\envs\big_data\lib\site-packages\numba\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more informatio

505 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
import math

@jit
def numba_score_2(scores):
    average = sum(scores) / len(scores)

    deviations = [(value - average) ** 2 for value in scores]      
    return math.sqrt((1 / (len(deviations) - 1) * sum(deviations)))

In [5]:
%%timeit
numba_score_2(df.Score.values.astype(int))

<ipython-input-4-357b6f70b1c5>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "numba_score_2" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-4-357b6f70b1c5>", line 5:
def numba_score_2(scores):
    average = sum(scores) / len(scores)
    ^

  @jit
c:\users\svw-b\envs\big_data\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "numba_score_2" was compiled in object mode without forceobj=True.

File "<ipython-input-4-357b6f70b1c5>", line 4:
@jit
def numba_score_2(scores):
^

  state.func_ir.loc))
c:\users\svw-b\envs\big_data\lib\site-packages\numba\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprec

2.85 s ± 58.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cython

In [2]:
%load_ext Cython

In [3]:
%%cython
cpdef float cython_standard_score_1(list reader):
    import math

    cdef list total = []

    for row in reader:
        total.append(int(row[6]))

    cdef float average = sum(total) / len(total)
    cdef list total2 = []
    for value in total:
        total2.append((value - average) ** 2)

    cdef float std_dev = math.sqrt((1 / (len(total2) -1) * sum(total2)))

    return std_dev

In [4]:
%%timeit
cython_standard_score_1(data)

221 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%cython
cpdef float cython_pandas_score_2(df): 
    import math
    
    deviations = (df.Score - df.Score.mean()) ** 2     
    return math.sqrt((1 / (len(deviations) - 1) * deviations.sum()))

In [5]:
%%timeit
cython_pandas_score_2(df)

9.41 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
%%cython
cpdef float cython_numpy_score_2(df): 
    import math
    
    deviations = (df.Score.values - df.Score.values.mean()) ** 2     
    return math.sqrt((1 / (len(deviations) - 1) * deviations.sum()))

In [4]:
%%timeit
cython_numpy_score_2(df)

5.42 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
%%cython
cpdef float cython_numpy_score_1(df):
    return df.Score.values.std()

In [4]:
%%timeit
cython_numpy_score_1(df)

3.54 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Experiment 3: Sorting it out (text)
Sort the summaries of the review on alphabetically ascending

#### Regular Python

In [1]:
with open('Reviews.csv', encoding="utf-8") as csvfile:
    import csv
    reader = csv.reader(csvfile)
    next(reader)
    data = [row for row in reader]

In [2]:
def python_standard_sorting_1(reader):
    summaries = []
    for row in reader:
        summaries.append(row[8])
    return sorted(summaries)

In [3]:
%%timeit
python_standard_sorting_1(data)

456 ms ± 5.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def python_standard_sorting_2(reader):
    return sorted([row[8] for row in reader])

In [4]:
%%timeit
python_standard_sorting_2(data)

425 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# def qsort(inlist):
#     if inlist == []: 
#         return []
#     else:
#         pivot = inlist[0]
#         lesser = qsort([x for x in inlist[1:] if x < pivot])
#         greater = qsort([x for x in inlist[1:] if x >= pivot])
#         return lesser + [pivot] + greater

In [9]:
# %%time
# qsort([row[8] for row in data])

#### Pandas

In [1]:
import pandas as pd
df = pd.read_csv("Reviews.csv")

In [2]:
def pandas_sorting_1(df):
    return df.Summary.sort_values()

In [3]:
%%timeit
pandas_sorting_1(df)

945 ms ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def pandas_sorting_2(df):
    return sorted(list(df.Summary.astype(str)))

In [3]:
%%timeit
pandas_sorting_2(df)

396 ms ± 8.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def pandas_sorting_3(df):
    return sorted(df.Summary.astype(str))

In [3]:
%%timeit
pandas_sorting_3(df)

386 ms ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numpy

In [2]:
def numpy_sorting_1(df):
    import numpy as np
    return np.sort(df.Summary.astype(str).values)

In [3]:
%%timeit
numpy_sorting_1(df)

732 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def numpy_sorting_2(df):
    return sorted(df.Summary.astype(str).values)

In [3]:
%%timeit
numpy_sorting_2(df)

353 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numba

In [2]:
from numba import jit

In [3]:
# From https://github.com/lprakash/Sorting-Algorithms/blob/master/sorts.ipynb
@jit
def quicksort(a, s, e):
    #print(a, s, e)
    #print(a.__repr__)
    if (e-s)==0:
        return 
    pivot = a[e-1]
    p1 = s
    p2 = e - 1
    while (p1 != p2):
        if (a[p1] > pivot):
            a[p2] = a[p1]
            a[p1] = a[p2-1]
            a[p2-1] = pivot
            p2 = p2 -1
        else: 
            p1+=1
    quicksort(a, s, p2)
    quicksort(a, p2+1, e)

In [4]:
def quicksortwrapper(a):
    quicksort(a, 0, len(a))

In [5]:
%%timeit
a = list([row[8] for row in data])
quicksort(a, 0, 4)

c:\users\svw-b\envs\big_data\lib\site-packages\numba\ir_utils.py:2041: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'a' of function 'quicksort'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-3-e903ddbe35be>", line 3:
@jit
def quicksort(a, s, e):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


1.27 s ± 7.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cython

In [2]:
%load_ext Cython

In [3]:
%%cython
cpdef list python_standard_sorting_2(list reader):
    return sorted([row[8] for row in reader])

In [4]:
%%timeit
python_standard_sorting_2(data)

390 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Experiment 3B: Sorting it out (int)
Sort the scores of the reviews

#### Regular Python

In [3]:
with open('Reviews.csv', encoding="utf-8") as csvfile:
    import csv
    reader = csv.reader(csvfile)
    next(reader)
    data = [row for row in reader]

In [2]:
def python_standard_sorting_1(reader):
    summaries = []
    for row in reader:
        summaries.append(row[6])
    return sorted(summaries)

In [3]:
%%timeit
python_standard_sorting_1(data)

128 ms ± 3.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
def python_standard_sorting_1b(reader):
    summaries = []
    for row in reader:
        summaries.append(int(row[6]))
    return sorted(summaries)

In [3]:
%%time
python_standard_sorting_1b(data)

Wall time: 223 ms


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [2]:
def python_standard_sorting_2(reader):
    return sorted([row[6] for row in reader])

In [4]:
%%timeit
python_standard_sorting_2(data)

105 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [470]:
# def qsort(inlist):
#     if inlist == []: 
#         return []
#     else:
#         pivot = inlist[0]
#         lesser = qsort([x for x in inlist[1:] if x < pivot])
#         greater = qsort([x for x in inlist[1:] if x >= pivot])
#         return lesser + [pivot] + greater

In [475]:
# %%time
# qsort([int(row[6]) for row in data])

#### Pandas

In [6]:
import pandas as pd
df = pd.read_csv("Reviews.csv")

In [2]:
def pandas_sorting_1(df):
    return df.Score.sort_values()

In [3]:
%%timeit
pandas_sorting_1(df)

37.6 ms ± 489 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
def pandas_sorting_2(df):
    return sorted(list(df.Score.astype(int)))

In [3]:
%%timeit
pandas_sorting_2(df)

80 ms ± 407 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
def pandas_sorting_3(df):
    return sorted(df.Score.astype(int))

In [3]:
%%timeit
pandas_sorting_3(df)

77.6 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Numpy

In [2]:
def numpy_sorting_1(df):
    import numpy as np
    return np.sort(df.Score.values)

In [3]:
%%timeit
numpy_sorting_1(df)

11.1 ms ± 64 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
def numpy_sorting_2(df):
    return sorted(df.Score.astype(str).values)

In [3]:
%%timeit
numpy_sorting_2(df)

391 ms ± 6.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numba

In [1]:
from numba import jit

In [3]:
# From https://github.com/lprakash/Sorting-Algorithms/blob/master/sorts.ipynb
@jit
def quicksort(a, s, e):
    #print(a, s, e)
    #print(a.__repr__)
    if (e-s)==0:
        return 
    pivot = a[e-1]
    p1 = s
    p2 = e - 1
    while (p1 != p2):
        if (a[p1] > pivot):
            a[p2] = a[p1]
            a[p1] = a[p2-1]
            a[p2-1] = pivot
            p2 = p2 -1
        else: 
            p1+=1
    quicksort(a, s, p2)
    quicksort(a, p2+1, e)

In [6]:
@jit
def sort(l):
    return sorted(l)

In [7]:
%%timeit
a = list([int(row[6]) for row in data])
sort(a)

<ipython-input-6-026d6d4df67e>:3: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'iterable' of function 'list_constructor.<locals>.list_impl'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\users\svw-b\envs\big_data\lib\site-packages\numba\targets\listobj.py", line 464:

    def list_impl(iterable):
    ^

  return sorted(l)
<ipython-input-6-026d6d4df67e>:3: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'iterable' of function 'ol_sorted.<locals>.impl'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\users\svw-b\envs\big_data\lib\site-packages\numba\targets\listobj.py", line 1108:



1.54 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def quicksortwrapper(a):
    quicksort(a, 0, len(a))

In [5]:
%%timeit
a = list([row[6] for row in data])
quicksort(a, 0, 4)

c:\users\svw-b\envs\big_data\lib\site-packages\numba\ir_utils.py:2041: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'a' of function 'quicksort'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-3-e903ddbe35be>", line 3:
@jit
def quicksort(a, s, e):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


1.29 s ± 50.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cython

In [1]:
%load_ext Cython

In [2]:
%%cython
cpdef list cython_standard_sorting_1(list reader):
    return sorted([row[6] for row in reader])

In [4]:
%%timeit
cython_standard_sorting_1(data)

78.9 ms ± 557 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Extra / Testing area

In [12]:
import random
listt = [random.randint(0, 100) for x in range(100000)]

In [39]:
%%time
new_list = []
for x in listt:
    new_list.append(x + 1)
new_list[0]

Wall time: 15 ms


86

In [40]:
%%time
[x + 1 for x in listt][0]

Wall time: 5.99 ms


86

In [36]:
def addition(n):
    return n + 1

In [41]:
%%time
list(map(lambda x: x + 1, listt))[0]

Wall time: 9.99 ms


86

In [4]:
%%timeit
import csv
with open('Reviews.csv', encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        for x in reader:
            pass

5.29 s ± 58.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
import pandas as pd
pd.read_csv("Reviews.csv")

3.68 s ± 82.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
import csv
test = open('Reviews.csv', encoding="utf8")    
reader = csv.reader(test)

In [27]:
def extra_test(reader):
    def parse_row_chocolate(row, word):
        if word in row[9].lower():
            return row[6]

    total = []
    for row in reader:
        result = parse_row_chocolate(row, "chocolate")
        if result:
            total.append((result))
    return total

In [26]:
%%time
extra_test(reader)

Wall time: 5.66 s


['5',
 '5',
 '5',
 '5',
 '1',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '3',
 '5',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '1',
 '1',
 '5',
 '1',
 '4',
 '5',
 '5',
 '3',
 '5',
 '5',
 '5',
 '2',
 '5',
 '4',
 '5',
 '5',
 '1',
 '3',
 '4',
 '5',
 '1',
 '5',
 '5',
 '5',
 '5',
 '3',
 '4',
 '5',
 '5',
 '5',
 '5',
 '1',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '1',
 '5',
 '5',
 '5',
 '1',
 '4',
 '5',
 '5',
 '1',
 '5',
 '4',
 '5',
 '5',
 '3',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '3',
 '5',
 '5',
 '4',
 '4',
 '4',
 '4',
 '5',
 '4',
 '5',
 '5',
 '4',
 '4',
 '5',
 '5',
 '1',
 '5',
 '1',
 '2',
 '5',
 '5',
 '5',
 '3',
 '5',
 '5',
 '5',
 '5',
 '3',
 '5',
 '5',
 '5',
 '4',
 '2',
 '4',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '1',
 '4',
 '5',
 '2',
 '2',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '3',
 '4',
 '5',
 '5',
 '5'

['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']


In [4]:
test.close()